# Two way ANOVA 

(week 11 - two way ANOVA)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.power as smp
import statsmodels.stats.proportion as smprop


### 1) Introduction and data example

To introduce two-way ANOVA we start by considering the following data (and do some data visuallisation)

In [ ]:
# Make pandas dataframe with grouped data:
data = pd.DataFrame({
    'value':      [2.8, 3.6, 3.4, 2.3, 5.5, 6.3, 6.1, 5.7, 5.8, 8.3, 6.9, 6.1], 
    'treatment':  ["A", "A", "A", "A", "B", "B", "B", "B", "C", "C", "C", "C"],
    'block':      ["1", "2", "3", "4", "1", "2", "3", "4", "1", "2", "3", "4"]})
data

In [ ]:
# Alternative way of typing the data:
data = pd.DataFrame({
    'value':      [2.8, 3.6, 3.4, 2.3, 5.5, 6.3, 6.1, 5.7, 5.8, 8.3, 6.9, 6.1], 
    'treatment':  np.repeat(["A", "B", "C"], 4),
    'block':      np.tile(["1", "2", "3", "4"], 3)})
data

In [ ]:
# Alternative way of typing the data (more like the book):

# OBS: in lecture copy paste from book p 329 to show issue with special characters

y = np.array([2.8, 3.6, 3.4, 2.3,
              5.5, 6.3, 6.1, 5.7,
              5.8, 8.3, 6.9, 6.1])
treatm = pd.Categorical([1, 1, 1, 1,
                         2, 2, 2, 2,
                         3, 3, 3, 3])
block = pd.Categorical([1, 2, 3, 4,
                        1, 2, 3, 4,
                        1, 2, 3, 4])
D = pd.DataFrame({'y': y, 'treatm': treatm, 'block': block})
D

Different ways of typing data result in the same kind of dataframe (table with data)

Be aware of copy-paste issues with special characters 

##### Visualising the data with box plots:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
data.boxplot("value", by="treatment", ax=axs[0], showmeans=True)
data.boxplot("value", by="block", ax=axs[1], showmeans=True)
fig.suptitle("Boxplots")

KAHOOT! (x2)

### 2) SST, SS(Tr) and SS(Bl)

In [ ]:
# Compute the overall mean and add to dataframe:
data['overall_mean'] = data["value"].mean()
data

In [ ]:
# compute the mean within each treatment-group and add to dataframe:
data['treatment_mean'] = data.groupby("treatment")['value'].transform('mean')
data

In [ ]:
# compute the mean within each person (block) and add to dataframe:
data['block_mean'] = data.groupby("block")['value'].transform('mean')
data

In [ ]:
# SST
SST = np.sum((data['value'] - data['overall_mean'])**2)
print(SST)

In [ ]:
# SS(Tr)
SSTr = np.sum((data['treatment_mean'] - data['overall_mean'])**2)
print(SSTr)

In [ ]:
# SS(Bl)
SSBl = np.sum((data['block_mean'] - data['overall_mean'])**2)
print(SSBl)

### 3) Estimate parameters $\mu$, $\alpha_i$, $\beta_j$ and $\sigma^2$ (MSE)

$Y_{ij} = \mu + \alpha_i + \beta_j + \epsilon_{ij}$

We start with residuals (and SSE):

In [ ]:
# alpha (for each Treatment group):
data["treatment_alpha"] = data["treatment_mean"] - data["overall_mean"]
data

In [ ]:
# beta (for each Block):
data["block_beta"] = data["block_mean"] - data["overall_mean"]
data

##### Residuals

In [ ]:
# calculate the residuals according to the model:
data['residual'] = data['value'] - (data['overall_mean']+data['treatment_alpha']+data['block_beta'])
data

In [ ]:
SSE = np.sum(data['residual']**2)
print(SSE)

In [ ]:
# sanity check for decomposition of variation (SST = SSTr + SSBl + SSE):
print(SST)
print(SSTr + SSBl + SSE)

In [ ]:
# estimating the residual variance:
MSE = SSE/((3-1)*(4-1))      # MSE = SSE /((k-1)*(l-1))
sigma = np.sqrt(MSE)
print(sigma)

## 4) F-test and ANOVA table

TReatment groups:

$F_{Tr}=\frac{SS(Tr)/(k-1)}{SSE/((k-1)(l-1))}$

In [ ]:
# Compute F-statistic (for treatment groups)
FTr = (SSTr/(3-1)) / (SSE/((3-1)*(4-1)))
print(FTr)

In [ ]:
# compute critical value:
stats.f.ppf(0.95, dfn = (3-1), dfd = (3-1)*(4-1) )

In [ ]:
# p-value
p_Tr = 1 - stats.f.cdf(FTr, dfn = (3-1), dfd = (3-1)*(4-1) )
print(p_Tr)

Blocks:

$F_{Bl}=\frac{SS(Bl)/(l-1)}{SSE/((k-1)(l-1))}$

In [ ]:
# Compute F-statistic (for block)
FBl = (SSBl/(4-1)) / (SSE/((3-1)*(4-1)))
print(FBl)

In [ ]:
# compute critical value:
stats.f.ppf(0.95, dfn = (4-1), dfd = (3-1)*(4-1) )

In [ ]:
# p-value
p_Bl = 1 - stats.f.cdf(FBl, dfn = (4-1), dfd = (3-1)*(4-1) )
print(p_Bl)

In [ ]:
# Make the ANOVA table:
fit = smf.ols("value ~ treatment + block", data=data).fit()
anova_table = sm.stats.anova_lm(fit)
print(anova_table)


In Python the ANOVA table does not have a row for "total sum of squares"

KAHOOT! (x2)

## 5) Model control

$Y_{ij} = \mu + \alpha_i + \beta_j + \epsilon_{ij}$

In [ ]:
# QQplot:
sm.qqplot(data["residual"], line='q',a=1/2)
plt.tight_layout()
plt.show()

In [ ]:
# Histogram:
plt.hist(data["residual"])
plt.tight_layout()
plt.show()

Normalfordelt?

Er de også uafhængige? (fx uafhængige af gruppering)

In [ ]:
# Plot residuals grouped by either treatment or block:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
data.boxplot("residual", by="treatment", showmeans=True, ax=axs[0])
data.boxplot("residual", by="block", showmeans=True, ax=axs[1])
fig.suptitle("Boxplots of residuals")
plt.show()

Hvordan vurderer vi det visuelt?

Er der "systematiske" tendenser (mønster)?

Hvad er trekanterne i plottet ovenfor (og hvorfor er de alle nul?)

## 6) Example from book

We do this together during the lecture

In [ ]:
D = pd.DataFrame({
'y': [22.5, 24.3, 24.9, 22.4,
21.5, 21.3, 23.9, 18.4,
22.2, 21.9, 21.7, 17.9],
'car': pd.Categorical([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]),
'tire': pd.Categorical([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]),
})


In [ ]:
D

In [ ]:
print(D['y'].mean())

In [ ]:
print(D.groupby("tire")['y'].mean())

In [ ]:
fig, ax = plt.subplots(ncols=2)
D.boxplot(column='y', by='tire', ax=ax[0],grid=False)
ax[0].set_title('Boxplots by tire')
D.boxplot(column='y', by='car', ax=ax[1],grid=False)
ax[1].set_title('Boxplots by car')

plt.tight_layout()
plt.show()

In [ ]:
fit = smf.ols('y ~ car + tire', data=D).fit()
anova = sm.stats.anova_lm(fit)
print(anova)

In [ ]:
D['residuals'] = fit.resid.values 
D

In [ ]:
plt.hist(D['residuals'])
plt.show()

In [ ]:
sm.qqplot(D['residuals'], line='q', a=1/2)
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=2)
D.boxplot(column='residuals', by='car', ax=ax[0],grid=False,  showmeans=True)
ax[0].set_title('Residuals by car')
D.boxplot(column='residuals', by='tire', ax=ax[1],grid=False,  showmeans=True)
ax[1].set_title('Residuals by tire')
plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
MSE = anova.loc['Residual', 'mean_sq']
print(MSE)

In [ ]:
D

In [ ]:
np.sum(D['residuals']**2)/(6)

In [ ]:
M = 3 * (3 - 1) / 2 
m = 4 
LSD_bonf = stats.t.ppf(1-(0.05/M)/2, 6) * np.sqrt(2*MSE/m)
print(LSD_bonf)

In [ ]:
print(D.groupby('tire',observed=True)['y'].mean())